In [1]:
%load_ext memory_profiler

## Load Dependancies 

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.svm import SVC,SVR

from sklearn import datasets
import scipy.stats as stats
import lightgbm as lgb
import xgboost as xgb

from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.feature_selection import RFE
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from pathlib import Path
import pandas as pd
import numpy as np

import dask
from dask.distributed import worker_client
from dask.distributed import Client,LocalCluster


import joblib
import numpy as np
from dask.distributed import Client

import joblib
from sklearn.datasets import load_digits
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC

In [3]:
client = Client(scheduler_file='scheduler.json')
client

Client Scheduler: tcp://10.6.21.67:8787 Dashboard: http://10.6.21.67:8791/status,Cluster Workers: 96 Cores: 96 Memory: 404.96 GB


In [4]:
dask.config.set(scheduler='threads')


## Load Data

In [5]:
root = Path('/scratch/ge3/sg4953') 

In [6]:
# read the csv file created
train = np.genfromtxt(str(root)+'/input_dataset.csv', delimiter=',')
train = train.astype(np.float32)
train = train[~np.isnan(train).any(axis=1)]
train = train[~(train==-9999.0).any(axis=1)]

X, y = train[:, 1:], train[:, :1]
y = y.ravel()


In [7]:
# Set training and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [10]:
# check for nodata values
# for i in range (train.shape[0]):
#     for j in range(train.shape[1]):
#         if train[i][j] == -9999.0:
#             print(i,j,train[i][j])
        

In [9]:
%%time
%%memit
#XGBoost

params={
    "max_depth":5, 
    "min_child_weight":2, 
    "eta": 1, 
    "colsample_bytree":0.8,  
    "eval_metric": "auc",
    'objective':'reg:squarederror'
}

clf = xgb.XGBModel(**params)


peak memory: 252.43 MiB, increment: 0.02 MiB
CPU times: user 88.6 ms, sys: 43.1 ms, total: 132 ms
Wall time: 281 ms


In [ ]:
%%time
%%memit
# Single Node

scores = cross_val_score(clf, X, y, cv=5,scoring='neg_mean_squared_error', n_jobs=-1)
print("MSE:"+ str(-scores.mean()))

In [10]:
%%time
%%memit
# Multiple Nodes 
with joblib.parallel_backend('dask'):
    scores = cross_val_score(clf, X, y, cv=5,scoring='neg_mean_squared_error')
print("MSE:"+ str(-scores.mean()))

MSE:0.1027763407677412
peak memory: 368.35 MiB, increment: 115.90 MiB
CPU times: user 231 ms, sys: 150 ms, total: 381 ms
Wall time: 1min 16s


In [ ]:
# Distributed Grid Search

In [11]:
from sklearn.model_selection import GridSearchCV

# Set the parameters by cross-validation
param_grid = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

xgb_model = xgb.XGBModel(
    learning_rate=0.02, 
    n_estimators=10, 
    objective="reg:squarederror"
)

grid_search = GridSearchCV(
    xgb_model,
    param_grid=param_grid,
    return_train_score=False,
    cv=3,
    scoring="precision",
    n_jobs=-1
)


In [14]:
# with joblib.parallel_backend('dask'):
grid_search.fit(X, y)

Task was destroyed but it is pending!
task: <Task pending coro=<DaskDistributedBackend._collect() running at /home/547/sg4953/.local/lib/python3.7/site-packages/joblib/_dask.py:198> wait_for=<Future pending cb=[<TaskWakeupMethWrapper object at 0x147f9133ad10>()]> cb=[IOLoop.add_future.<locals>.<lambda>() at /apps/pangeo/2020.05/lib/python3.7/site-packages/tornado/ioloop.py:690]>
/home/547/sg4953/.local/lib/python3.7/site-packages/sklearn/model_selection/_search.py:921: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan

GridSearchCV(cv=3,
             estimator=XGBModel(learning_rate=0.02, n_estimators=10,
                                objective='reg:squarederror'),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.6, 0.8, 1.0],
                         'gamma': [0.5, 1, 1.5, 2, 5], 'max_depth': [3, 4, 5],
                         'min_child_weight': [1, 5, 10],
                         'subsample': [0.6, 0.8, 1.0]},
             scoring='precision')

In [13]:
pd.DataFrame(grid_search.cv_results_).head()

AttributeError: 'GridSearchCV' object has no attribute 'cv_results_'

In [ ]:
from skopt import BayesSearchCV
from sklearn.svm import SVC

param_grid = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

xgb_model = xgb.XGBModel(
    learning_rate=0.02, 
    n_estimators=50, 
    objective=reg:squarederror
)


bayessearchcv = BayesSearchCV(
    xgb_model,
    search_spaces=param_grid,
    n_iter=10,
    cv=3,
    n_jobs=-1
)

# callback handler
def on_step(optim_result):
    score = searchcv.best_score_
    print("best score: %s" % score)
    if score >= 0.98:
        print('Interrupting!')
        return True


bayessearchcv.fit(X, y, callback=on_step)